In [50]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from scipy import stats
# import tensorflow as tf

In [51]:
# Import our mental health dataset and create a dataframe
mental_df = pd.read_csv('osmi 2016.csv')
mental_df

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",...,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,NaN,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,...,Not applicable to me,Not applicable to me,39,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Sometimes
1,0,25-Jun,1.0,NaN,No,Yes,Yes,Yes,Yes,Somewhat easy,...,Rarely,Sometimes,29,male,United States of America,Illinois,United States of America,Illinois,Back-end Developer|Front-end Developer,Never
2,0,25-Jun,1.0,NaN,No,NaN,No,No,I don't know,Neither easy nor difficult,...,Not applicable to me,Not applicable to me,38,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Always
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sometimes,Sometimes,43,male,United Kingdom,NaN,United Kingdom,NaN,Supervisor/Team Lead,Sometimes
4,0,25-Jun,0.0,1.0,Yes,Yes,No,No,No,Neither easy nor difficult,...,Sometimes,Sometimes,43,Female,United States of America,Illinois,United States of America,Illinois,Executive Leadership|Supervisor/Team Lead|Dev ...,Sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Not applicable to me,Not applicable to me,34,Female,United States of America,New York,United States of America,New York,Other,Sometimes
1429,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Sometimes,Often,56,MALE,United States of America,California,Afghanistan,NaN,Support,Sometimes
1430,0,100-500,1.0,NaN,Yes,Yes,Yes,Yes,I don't know,Somewhat difficult,...,Rarely,Sometimes,52,Male,United States of America,Georgia,United States of America,Georgia,Back-end Developer,Sometimes
1431,0,100-500,0.0,1.0,I don't know,I am not sure,No,Yes,I don't know,Somewhat difficult,...,Sometimes,Often,30,Female,United States of America,Nebraska,United States of America,Nebraska,DevOps/SysAdmin,Sometimes


In [52]:
# Replace all NaN values with 0
mental_df = mental_df.replace(to_replace=pd.np.nan, value=0)

/tmp/ipykernel_3342/1301895068.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  mental_df = mental_df.replace(to_replace=pd.np.nan, value=0)


In [53]:
mental_df

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",...,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,0.0,Not eligible for coverage / N/A,0,No,No,I don't know,Very easy,...,Not applicable to me,Not applicable to me,39,Male,United Kingdom,0,United Kingdom,0,Back-end Developer,Sometimes
1,0,25-Jun,1.0,0.0,No,Yes,Yes,Yes,Yes,Somewhat easy,...,Rarely,Sometimes,29,male,United States of America,Illinois,United States of America,Illinois,Back-end Developer|Front-end Developer,Never
2,0,25-Jun,1.0,0.0,No,0,No,No,I don't know,Neither easy nor difficult,...,Not applicable to me,Not applicable to me,38,Male,United Kingdom,0,United Kingdom,0,Back-end Developer,Always
3,1,0,0.0,0.0,0,0,0,0,0,0,...,Sometimes,Sometimes,43,male,United Kingdom,0,United Kingdom,0,Supervisor/Team Lead,Sometimes
4,0,25-Jun,0.0,1.0,Yes,Yes,No,No,No,Neither easy nor difficult,...,Sometimes,Sometimes,43,Female,United States of America,Illinois,United States of America,Illinois,Executive Leadership|Supervisor/Team Lead|Dev ...,Sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,0,0.0,0.0,0,0,0,0,0,0,...,Not applicable to me,Not applicable to me,34,Female,United States of America,New York,United States of America,New York,Other,Sometimes
1429,1,0,0.0,0.0,0,0,0,0,0,0,...,Sometimes,Often,56,MALE,United States of America,California,Afghanistan,0,Support,Sometimes
1430,0,100-500,1.0,0.0,Yes,Yes,Yes,Yes,I don't know,Somewhat difficult,...,Rarely,Sometimes,52,Male,United States of America,Georgia,United States of America,Georgia,Back-end Developer,Sometimes
1431,0,100-500,0.0,1.0,I don't know,I am not sure,No,Yes,I don't know,Somewhat difficult,...,Sometimes,Often,30,Female,United States of America,Nebraska,United States of America,Nebraska,DevOps/SysAdmin,Sometimes


In [54]:
mental_df = mental_df.apply(lambda x: x.astype(str).str.lower())

In [55]:
mental_df.shape

(1433, 63)

In [56]:
mental_df

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",...,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,0.0,not eligible for coverage / n/a,0,no,no,i don't know,very easy,...,not applicable to me,not applicable to me,39,male,united kingdom,0,united kingdom,0,back-end developer,sometimes
1,0,25-jun,1.0,0.0,no,yes,yes,yes,yes,somewhat easy,...,rarely,sometimes,29,male,united states of america,illinois,united states of america,illinois,back-end developer|front-end developer,never
2,0,25-jun,1.0,0.0,no,0,no,no,i don't know,neither easy nor difficult,...,not applicable to me,not applicable to me,38,male,united kingdom,0,united kingdom,0,back-end developer,always
3,1,0,0.0,0.0,0,0,0,0,0,0,...,sometimes,sometimes,43,male,united kingdom,0,united kingdom,0,supervisor/team lead,sometimes
4,0,25-jun,0.0,1.0,yes,yes,no,no,no,neither easy nor difficult,...,sometimes,sometimes,43,female,united states of america,illinois,united states of america,illinois,executive leadership|supervisor/team lead|dev ...,sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,0,0.0,0.0,0,0,0,0,0,0,...,not applicable to me,not applicable to me,34,female,united states of america,new york,united states of america,new york,other,sometimes
1429,1,0,0.0,0.0,0,0,0,0,0,0,...,sometimes,often,56,male,united states of america,california,afghanistan,0,support,sometimes
1430,0,100-500,1.0,0.0,yes,yes,yes,yes,i don't know,somewhat difficult,...,rarely,sometimes,52,male,united states of america,georgia,united states of america,georgia,back-end developer,sometimes
1431,0,100-500,0.0,1.0,i don't know,i am not sure,no,yes,i don't know,somewhat difficult,...,sometimes,often,30,female,united states of america,nebraska,united states of america,nebraska,devops/sysadmin,sometimes


In [57]:
# Replace "yes" with 1, "no" with -1, and "I don't know" with 0
mental_df = mental_df.replace({'yes': 1, 'no': -1, "i don't know": 0})

In [58]:
# Print out gender counts
gender_counts = mental_df['What is your gender?'].value_counts()
gender_counts

male                                                                                                                                                             860
female                                                                                                                                                           248
m                                                                                                                                                                165
f                                                                                                                                                                 61
male                                                                                                                                                              13
female                                                                                                                                                            12
woman     

In [59]:
# Determine which values to replace
replace_genders = list(gender_counts[gender_counts < 20].index)

# Replace in DataFrame
for gender in replace_genders:
    mental_df['What is your gender?'] = mental_df['What is your gender?'].replace(gender,"Other")

In [60]:
# Print out gender counts
gender_counts = mental_df['What is your gender?'].value_counts()
gender_counts

male      860
female    248
m         165
Other      99
f          61
Name: What is your gender?, dtype: int64

In [61]:
# Print out country counts
country_counts = mental_df['What country do you live in?'].value_counts()
country_counts

united states of america    840
united kingdom              180
canada                       78
germany                      58
netherlands                  48
australia                    35
sweden                       19
france                       16
ireland                      15
switzerland                  10
brazil                       10
new zealand                   9
india                         9
russia                        9
bulgaria                      7
finland                       7
denmark                       7
italy                         5
belgium                       5
south africa                  4
romania                       4
spain                         4
poland                        4
austria                       4
norway                        3
pakistan                      3
chile                         3
czech republic                3
japan                         2
israel                        2
other                         2
bosnia a

In [62]:
# Determine which values to replace
replace_countries = list(country_counts[country_counts < 20].index)

# Replace in DataFrame
for country in replace_countries:
    mental_df['What country do you live in?'] = mental_df['What country do you live in?'].replace(country,"Other")

In [63]:
# Print out country counts
country_counts = mental_df['What country do you live in?'].value_counts()
country_counts

united states of america    840
Other                       194
united kingdom              180
canada                       78
germany                      58
netherlands                  48
australia                    35
Name: What country do you live in?, dtype: int64

In [64]:
# Print out country counts
country_counts = mental_df['What country do you work in?'].value_counts()
country_counts

united states of america    851
united kingdom              183
canada                       74
germany                      58
netherlands                  47
australia                    34
sweden                       20
ireland                      15
france                       14
switzerland                  10
brazil                       10
india                         9
russia                        9
new zealand                   9
denmark                       7
bulgaria                      7
finland                       7
belgium                       5
south africa                  4
poland                        4
austria                       4
czech republic                3
italy                         3
chile                         3
norway                        3
romania                       3
spain                         3
other                         2
bosnia and herzegovina        2
afghanistan                   2
pakistan                      2
israel  

In [65]:
# Determine which values to replace
replace_countries = list(country_counts[country_counts < 20].index)

# Replace in DataFrame
for country in replace_countries:
    mental_df['What country do you work in?'] = mental_df['What country do you work in?'].replace(country,"Other")

In [66]:
mental_df


,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",...,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,0.0,not eligible for coverage / n/a,0,-1,-1,0,very easy,...,not applicable to me,not applicable to me,39,male,united kingdom,0,united kingdom,0,back-end developer,sometimes
1,0,25-jun,1.0,0.0,-1,1,1,1,1,somewhat easy,...,rarely,sometimes,29,male,united states of america,illinois,united states of america,illinois,back-end developer|front-end developer,never
2,0,25-jun,1.0,0.0,-1,0,-1,-1,0,neither easy nor difficult,...,not applicable to me,not applicable to me,38,Other,united kingdom,0,united kingdom,0,back-end developer,always
3,1,0,0.0,0.0,0,0,0,0,0,0,...,sometimes,sometimes,43,male,united kingdom,0,united kingdom,0,supervisor/team lead,sometimes
4,0,25-jun,0.0,1.0,1,1,-1,-1,-1,neither easy nor difficult,...,sometimes,sometimes,43,female,united states of america,illinois,united states of america,illinois,executive leadership|supervisor/team lead|dev ...,sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,0,0.0,0.0,0,0,0,0,0,0,...,not applicable to me,not applicable to me,34,Other,united states of america,new york,united states of america,new york,other,sometimes
1429,1,0,0.0,0.0,0,0,0,0,0,0,...,sometimes,often,56,male,united states of america,california,Other,0,support,sometimes
1430,0,100-500,1.0,0.0,1,1,1,1,0,somewhat difficult,...,rarely,sometimes,52,male,united states of america,georgia,united states of america,georgia,back-end developer,sometimes
1431,0,100-500,0.0,1.0,0,i am not sure,-1,1,0,somewhat difficult,...,sometimes,often,30,female,united states of america,nebraska,united states of america,nebraska,devops/sysadmin,sometimes


In [67]:
mental_df.dropna(inplace=True)

In [68]:
mental_df.shape

(1433, 63)

In [69]:
mental_df.replace(",", "")

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",...,"If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?","If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?",What is your age?,What is your gender?,What country do you live in?,What US state or territory do you live in?,What country do you work in?,What US state or territory do you work in?,Which of the following best describes your work position?,Do you work remotely?
0,0,26-100,1.0,0.0,not eligible for coverage / n/a,0,-1,-1,0,very easy,...,not applicable to me,not applicable to me,39,male,united kingdom,0,united kingdom,0,back-end developer,sometimes
1,0,25-jun,1.0,0.0,-1,1,1,1,1,somewhat easy,...,rarely,sometimes,29,male,united states of america,illinois,united states of america,illinois,back-end developer|front-end developer,never
2,0,25-jun,1.0,0.0,-1,0,-1,-1,0,neither easy nor difficult,...,not applicable to me,not applicable to me,38,Other,united kingdom,0,united kingdom,0,back-end developer,always
3,1,0,0.0,0.0,0,0,0,0,0,0,...,sometimes,sometimes,43,male,united kingdom,0,united kingdom,0,supervisor/team lead,sometimes
4,0,25-jun,0.0,1.0,1,1,-1,-1,-1,neither easy nor difficult,...,sometimes,sometimes,43,female,united states of america,illinois,united states of america,illinois,executive leadership|supervisor/team lead|dev ...,sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1,0,0.0,0.0,0,0,0,0,0,0,...,not applicable to me,not applicable to me,34,Other,united states of america,new york,united states of america,new york,other,sometimes
1429,1,0,0.0,0.0,0,0,0,0,0,0,...,sometimes,often,56,male,united states of america,california,Other,0,support,sometimes
1430,0,100-500,1.0,0.0,1,1,1,1,0,somewhat difficult,...,rarely,sometimes,52,male,united states of america,georgia,united states of america,georgia,back-end developer,sometimes
1431,0,100-500,0.0,1.0,0,i am not sure,-1,1,0,somewhat difficult,...,sometimes,often,30,female,united states of america,nebraska,united states of america,nebraska,devops/sysadmin,sometimes


In [70]:
mental_df.to_csv('cleaned_data.csv', index=False)

In [71]:
print("create table cleaned_data (")
for column in mental_df.columns: 
    print("=", column.replace(" ", "_").upper(), "% varchar,")

create table cleaned_data (
= ARE_YOU_SELF-EMPLOYED? % varchar,
= HOW_MANY_EMPLOYEES_DOES_YOUR_COMPANY_OR_ORGANIZATION_HAVE? % varchar,
= IS_YOUR_EMPLOYER_PRIMARILY_A_TECH_COMPANY/ORGANIZATION? % varchar,
= IS_YOUR_PRIMARY_ROLE_WITHIN_YOUR_COMPANY_RELATED_TO_TECH/IT? % varchar,
= DOES_YOUR_EMPLOYER_PROVIDE_MENTAL_HEALTH_BENEFITS_AS_PART_OF_HEALTHCARE_COVERAGE? % varchar,
= DO_YOU_KNOW_THE_OPTIONS_FOR_MENTAL_HEALTH_CARE_AVAILABLE_UNDER_YOUR_EMPLOYER-PROVIDED_COVERAGE? % varchar,
= HAS_YOUR_EMPLOYER_EVER_FORMALLY_DISCUSSED_MENTAL_HEALTH_(FOR_EXAMPLE,_AS_PART_OF_A_WELLNESS_CAMPAIGN_OR_OTHER_OFFICIAL_COMMUNICATION)? % varchar,
= DOES_YOUR_EMPLOYER_OFFER_RESOURCES_TO_LEARN_MORE_ABOUT_MENTAL_HEALTH_CONCERNS_AND_OPTIONS_FOR_SEEKING_HELP? % varchar,
= IS_YOUR_ANONYMITY_PROTECTED_IF_YOU_CHOOSE_TO_TAKE_ADVANTAGE_OF_MENTAL_HEALTH_OR_SUBSTANCE_ABUSE_TREATMENT_RESOURCES_PROVIDED_BY_YOUR_EMPLOYER? % varchar,
= IF_A_MENTAL_HEALTH_ISSUE_PROMPTED_YOU_TO_REQUEST_A_MEDICAL_LEAVE_FROM_WORK,_ASKING_FOR_THA

Bad pipe message: %s [b"o\x9eh\x8f\x8a\xc6\n\xfd\xfd(\xcb\xcbe\x7fBZ\t\xd0\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d"]
Bad pipe message: %s [b'\xb0}\x83\xa1\x80j\xb6Nt8\x04\x91', b"\x8e\xc2\xe2\x03\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0s\xc0w\x00\xc4\x00\xc3\xc0#\xc0'\x00g\x00@\xc0r\xc0v\x00\xbe\x00\xbd\xc0\n\xc0\x14\x009\x008\x00\x88\x00\x87\xc0\t\xc0\x13\x003\x002\x00\x9a\x00\x99\x00E\x00D\xc0\x07\xc0\x11\xc0\x08\xc0\x12\x00\x16\x00\x13\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00\xc0\x00<\x00\xba\x005\x00\x84\x00/\x00\x96\x00A\x00\x05\x00\n\x00\xf